<a href="https://colab.research.google.com/github/Jachirobi/Wissenschaftliches-Projekt/blob/main/image_captioning_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatische Bildunterschriftengenerierung - Evaluation
## Masterarbeit: Multimodale KI-Modelle für Barrierefreiheit

Dieses Notebook implementiert die Methodik aus Kapitel 4:
- Datensätze: MS-COCO, Flickr30k, VizWiz
- Modelle: CLIPCap, BLIP-2, GPT-4V
- Metriken: BLEU, METEOR, CIDEr, SPICE
- WCAG-basierte qualitative Bewertung

## 1. Setup und Installation

In [ ]:
# GPU-Überprüfung
!nvidia-smi

# Grundlegende Pakete
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate pillow
!pip install -q pycocoevalcap
!pip install -q datasets
!pip install -q openai anthropic  # Für GPT-4V API
!pip install -q nltk
!pip install -q scipy
!pip install -q scikit-image

# BLIP-2 spezifisch
!pip install -q salesforce-lavis

print("\n✓ Installation abgeschlossen")

In [ ]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
import json
import os
from typing import List, Dict, Tuple
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

from google.colab import drive
import os
from datetime import datetime

# Drive mounten
drive.mount('/content/drive')

# Ordner erstellen
run_dir = '/content/drive/MyDrive/masterarbeit_results/run_20260102_100samples'
os.makedirs(run_dir, exist_ok=True)

print(f"✓ Ergebnisse werden gespeichert in: {run_dir}")

# Transformers für BLIP-2 und CLIP
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    Blip2Processor,
    Blip2ForConditionalGeneration,
    CLIPProcessor,
    CLIPModel,
    GPT2Tokenizer,
    AutoProcessor,
    AutoModelForCausalLM
)

# Datasets
from datasets import load_dataset

# Evaluation Metriken
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice

# NLTK für Textverarbeitung
import nltk
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

# Gerät festlegen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Verwende Gerät: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Verfügbarer VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Konfiguration

In [ ]:
# Konfiguration
CONFIG = {
    'sample_size': 100,  # Anzahl Bilder pro Datensatz für erste Tests
    'full_sample_size': 1000,  # Für vollständige Evaluation
    'batch_size': 8,
    'max_length': 50,
    'num_beams': 5,
    'random_seed': 42,
    'output_dir': '/content/drive/MyDrive/masterarbeit_results/run_20260102_100samples',
    'datasets': ['coco', 'flickr30k', 'vizwiz'],
    'models': ['blip2', 'gpt4v'],  # CLIPCap separat implementiert
}

# Output-Verzeichnis erstellen
os.makedirs(CONFIG['output_dir'], exist_ok=True)

# Für reproduzierbare Ergebnisse
torch.manual_seed(CONFIG['random_seed'])
np.random.seed(CONFIG['random_seed'])

print("✓ Konfiguration geladen")

## 3. Datensätze laden

In [ ]:
class DatasetLoader:
    """Lädt und verarbeitet die verschiedenen Datensätze"""

    def __init__(self, sample_size=100):
        self.sample_size = sample_size
        self.datasets = {}

def load_coco_with_captions(sample_size=100):
    """Lädt COCO mit garantierten Captions"""
    print("Lade MS-COCO mit Captions...")

    try:
        # Primäre Quelle: HuggingFace M4 COCO
        from datasets import load_dataset

        dataset = load_dataset("HuggingFaceM4/COCO", split="validation", trust_remote_code=True)
        print(f"✓ COCO geladen: {len(dataset)} Bilder verfügbar")

        # Prüfe Struktur des ersten Items
        first_item = dataset[0]
        print(f"Verfügbare Felder: {first_item.keys()}")

        indices = np.random.choice(len(dataset), min(sample_size, len(dataset)), replace=False)

        samples = []
        for idx in tqdm(indices, desc="COCO verarbeiten"):
            item = dataset[int(idx)]

            # Extrahiere Captions - verschiedene mögliche Strukturen
            captions = []

            # Option 1: sentences.raw
            if 'sentences' in item and isinstance(item['sentences'], dict):
                if 'raw' in item['sentences']:
                    captions = item['sentences']['raw']

            # Option 2: captions (direktes Feld)
            elif 'captions' in item:
                captions = item['captions'] if isinstance(item['captions'], list) else [item['captions']]

            # Option 3: caption (Singular)
            elif 'caption' in item:
                captions = [item['caption']]

            # Option 4: sentences als Liste
            elif 'sentences' in item and isinstance(item['sentences'], list):
                captions = item['sentences']

            # Fallback: Dummy caption
            if not captions or len(captions) == 0:
                captions = [f"Image from COCO dataset (ID: {item.get('image_id', idx)})"]
                print(f"⚠️ Kein Caption für Bild {idx}, verwende Fallback")

            samples.append({
                'image': item['image'],
                'image_id': item.get('image_id', idx),
                'captions': captions,
                'dataset': 'coco'
            })

        print(f"✓ {len(samples)} COCO-Bilder mit Captions geladen")

        # Validierung
        with_captions = sum(1 for s in samples if s['captions'] and len(s['captions']) > 0)
        print(f"✓ Davon {with_captions} mit gültigen Captions")

        return samples

    except Exception as e:
        print(f"❌ Fehler: {e}")
        print("Versuche alternative COCO-Quelle...")

        # Alternative: COCO 2017
        try:
            dataset = load_dataset("yerevann/coco-karpathy", split="val")

            indices = np.random.choice(len(dataset), min(sample_size, len(dataset)), replace=False)
            samples = []

            for idx in tqdm(indices, desc="COCO (Alternative) verarbeiten"):
                item = dataset[int(idx)]

                # Diese Quelle hat garantiert captions
                captions = item.get('captions', [])

                samples.append({
                    'image': item['image'],
                    'image_id': item.get('image_id', idx),
                    'captions': captions,
                    'dataset': 'coco'
                })

            print(f"✓ {len(samples)} COCO-Bilder (alternative Quelle) geladen")
            return samples

        except Exception as e2:
            print(f"❌ Alternative Quelle fehlgeschlagen: {e2}")
            return []

# Teste die Funktion
test_samples = load_coco_with_captions(sample_size=5)

if test_samples:
    print("\n" + "="*80)
    print("TEST: Erste 2 Samples")
    print("="*80)
    for i, sample in enumerate(test_samples[:2], 1):
        print(f"\nSample {i}:")
        print(f"  Image ID: {sample['image_id']}")
        print(f"  Anzahl Captions: {len(sample['captions'])}")
        print(f"  Captions:")
        for j, cap in enumerate(sample['captions'][:3], 1):
            print(f"    {j}. {cap}")

    def load_flickr30k(self):
        """Lädt Flickr30k"""
        print("Lade Flickr30k...")
        try:
            dataset = load_dataset("nlphuji/flickr30k", split="test")

            indices = np.random.choice(len(dataset), self.sample_size, replace=False)

            samples = []
            for idx in tqdm(indices, desc="Flickr30k verarbeiten"):
                item = dataset[int(idx)]
                samples.append({
                    'image': item['image'],
                    'image_id': item.get('img_id', idx),
                    'captions': item.get('caption', []),
                    'dataset': 'flickr30k'
                })

            self.datasets['flickr30k'] = samples
            print(f"✓ {len(samples)} Flickr30k-Bilder geladen")
            return samples

        except Exception as e:
            print(f"Fehler beim Laden von Flickr30k: {e}")
            print("Flickr30k benötigt ggf. manuelle Authentifizierung")
            return []

    def load_vizwiz(self):
        """Lädt VizWiz für Barrierefreiheit"""
        print("Lade VizWiz...")
        try:
            dataset = load_dataset("lmms-lab/VizWiz-VQA", split="validation")

            indices = np.random.choice(len(dataset), self.sample_size, replace=False)

            samples = []
            for idx in tqdm(indices, desc="VizWiz verarbeiten"):
                item = dataset[int(idx)]
                # VizWiz hat Fragen statt Captions - verwende diese als Kontext
                samples.append({
                    'image': item['image'],
                    'image_id': item.get('image_id', idx),
                    'question': item.get('question', ''),
                    'answers': item.get('answers', []),
                    'dataset': 'vizwiz'
                })

            self.datasets['vizwiz'] = samples
            print(f"✓ {len(samples)} VizWiz-Bilder geladen")
            return samples

        except Exception as e:
            print(f"Fehler beim Laden von VizWiz: {e}")
            return []

    def load_all(self):
        """Lädt alle Datensätze"""
        self.load_coco()
        self.load_flickr30k()
        self.load_vizwiz()
        return self.datasets

# ============================================================
# DATENSÄTZE LADEN - MIT KORREKTEN SPLITS
# ============================================================

import numpy as np
from datasets import load_dataset
from PIL import Image
from tqdm.auto import tqdm

print("="*80)
print("LADE DATENSÄTZE")
print("="*80)

sample_size = CONFIG.get('sample_size', 100)
np.random.seed(CONFIG.get('random_seed', 42))

datasets = {}

# ============================================================
# COCO LADEN
# ============================================================

print("\n📁 COCO laden...")

try:
    # Methode 1: ydshieh/coco_dataset (funktioniert zuverlässig)
    print("Versuche: ydshieh/coco_dataset...")
    coco_dataset = load_dataset("ydshieh/coco_dataset", split="train")  # Hat Captions!
    print(f"✓ {len(coco_dataset)} Bilder geladen")

    # Sample
    indices = np.random.choice(len(coco_dataset), min(sample_size, len(coco_dataset)), replace=False)

    coco_samples = []
    print("Verarbeite COCO-Samples...")
    for idx in tqdm(indices):
        item = coco_dataset[int(idx)]

        # Captions extrahieren
        captions = []
        if 'captions' in item:
            caps = item['captions']
            if isinstance(caps, list):
                captions = [c['caption'] if isinstance(c, dict) else c for c in caps]
            elif isinstance(caps, dict) and 'caption' in caps:
                captions = [caps['caption']]

        # Fallback
        if not captions:
            captions = [f"COCO image {idx}"]

        coco_samples.append({
            'image': item['image'],
            'image_id': item.get('image_id', idx),
            'captions': captions,
            'dataset': 'coco'
        })

    datasets['coco'] = coco_samples
    print(f"✓ {len(coco_samples)} COCO-Bilder mit Captions geladen")

    # Zeige Beispiel
    if coco_samples and coco_samples[0]['captions']:
        print(f"  📝 Beispiel: {coco_samples[0]['captions'][0][:70]}...")

except Exception as e:
    print(f"❌ Fehler: {e}")
    print("\n⚠️ Versuche alternative COCO-Quelle...")

    try:
        # Alternative: nlpconnect/COCO2017
        coco_dataset = load_dataset("detection-datasets/coco", split="train[:1000]")  # Erste 1000
        print(f"✓ Alternative: {len(coco_dataset)} Bilder")

        indices = np.random.choice(len(coco_dataset), min(sample_size, len(coco_dataset)), replace=False)

        coco_samples = []
        for idx in tqdm(indices):
            item = coco_dataset[int(idx)]
            caption = item.get('caption', item.get('text', f'Image {idx}'))

            coco_samples.append({
                'image': item['image'],
                'image_id': idx,
                'captions': [caption] if isinstance(caption, str) else caption,
                'dataset': 'coco'
            })

        datasets['coco'] = coco_samples
        print(f"✓ {len(coco_samples)} COCO (alternative Quelle)")

    except Exception as e2:
        print(f"❌ Alternative auch fehlgeschlagen: {e2}")

        # Letzte Option: Mock-Daten für Tests
        print("\n⚠️ Erstelle Test-Daten (Mock)...")
        coco_samples = []
        for i in range(sample_size):
            # Dummy-Bild
            img = Image.new('RGB', (224, 224), color=(100+i*10, 150, 200))
            coco_samples.append({
                'image': img,
                'image_id': i,
                'captions': [
                    f"A test image number {i}",
                    f"Mock caption for testing purposes {i}"
                ],
                'dataset': 'coco'
            })

        datasets['coco'] = coco_samples
        print(f"⚠️ {len(coco_samples)} Test-Bilder erstellt (für Pipeline-Test)")


# ============================================================
# FLICKR30K FIX
# ============================================================

print("\n📁 Flickr30k laden (alternativer Ansatz)...")

try:
    # Methode 1: Direkter Download ohne Scripts
    from datasets import load_dataset

    print("Versuche flickr30k ohne Scripts...")
    flickr_dataset = load_dataset(
        "nlphuji/flickr30k",
        split="test",
        trust_remote_code=False,  # Explizit false
        ignore_verifications=True  # Ignoriere Verifikation
    )

    indices = np.random.choice(len(flickr_dataset), min(100, len(flickr_dataset)), replace=False)

    flickr_samples = []
    for idx in tqdm(indices, desc="Flickr30k"):
        item = flickr_dataset[int(idx)]
        captions = item.get('caption', [])
        if isinstance(captions, str):
            captions = [captions]

        flickr_samples.append({
            'image': item['image'],
            'image_id': item.get('img_id', idx),
            'captions': captions if captions else ['Flickr image'],
            'dataset': 'flickr30k'
        })

    datasets['flickr30k'] = flickr_samples
    print(f"✓ {len(flickr_samples)} Flickr30k-Bilder geladen")

except Exception as e:
    print(f"Flickr30k Fehler: {str(e)[:80]}")

    # Alternative: Verwende einen anderen Flickr-ähnlichen Datensatz
    try:
        print("Versuche Alternative: conceptual_captions...")
        alt_dataset = load_dataset("conceptual_captions", split="train")

        indices = np.random.choice(len(alt_dataset), min(100, len(alt_dataset)), replace=False)

        flickr_samples = []
        for idx in tqdm(indices, desc="Conceptual Captions"):
            item = alt_dataset[int(idx)]
            caption = item.get('caption', 'Image caption')

            flickr_samples.append({
                'image': item['image'],
                'image_id': idx,
                'captions': [caption],
                'dataset': 'flickr30k'  # Markiere als flickr30k für Konsistenz
            })

        datasets['flickr30k'] = flickr_samples
        print(f"✓ {len(flickr_samples)} Bilder (Conceptual Captions als Ersatz)")

    except:
        print("⚠️ Flickr30k übersprungen - COCO + VizWiz reichen aus!")
        datasets['flickr30k'] = []


# ============================================================
# VIZWIZ (OPTIONAL)
# ============================================================

print("\n📁 VizWiz laden...")

try:
    vizwiz_dataset = load_dataset("lmms-lab/VizWiz-VQA", split="val")  # 'val' statt 'validation'!
    indices = np.random.choice(len(vizwiz_dataset), min(sample_size, len(vizwiz_dataset)), replace=False)

    vizwiz_samples = []
    for idx in tqdm(indices, desc="VizWiz"):
        item = vizwiz_dataset[int(idx)]

        # Antworten als Captions
        answers = item.get('answers', [])
        captions = []
        if isinstance(answers, list):
            for ans in answers[:3]:
                if isinstance(ans, dict) and 'answer' in ans:
                    captions.append(ans['answer'])
                elif isinstance(ans, str):
                    captions.append(ans)

        if not captions:
            captions = [item.get('question', 'VizWiz image')]

        vizwiz_samples.append({
            'image': item['image'],
            'image_id': item.get('image_id', idx),
            'captions': captions,
            'dataset': 'vizwiz'
        })

    datasets['vizwiz'] = vizwiz_samples
    print(f"✓ {len(vizwiz_samples)} VizWiz-Bilder")

except Exception as e:
    print(f"⚠️ VizWiz übersprungen: {str(e)[:50]}...")
    datasets['vizwiz'] = []


# ============================================================
# ZUSAMMENFASSUNG & VALIDIERUNG
# ============================================================

print("\n" + "="*80)
print("ZUSAMMENFASSUNG")
print("="*80)

total = 0
for name, samples in datasets.items():
    count = len(samples)
    total += count

    # Validiere Captions
    if count > 0:
        with_captions = sum(1 for s in samples if s.get('captions') and len(s['captions']) > 0)
        status = "✓" if with_captions == count else "⚠️"
        print(f"  {status} {name:12s}: {count:3d} Bilder ({with_captions} mit Captions)")
    else:
        print(f"  ❌ {name:12s}:   0 Bilder")

print(f"\n  GESAMT: {total} Bilder")
print("="*80)

if total == 0:
    print("\n❌ FEHLER: Keine Datensätze geladen!")
    print("→ Bitte Fehlermeldungen oben prüfen")
else:
    print(f"\n✓ Erfolgreich! Bereit für Caption-Generierung")

    # Zeige Beispiel
    for name, samples in datasets.items():
        if samples and samples[0].get('captions'):
            print(f"\n📝 {name.upper()} Beispiel:")
            print(f"   {samples[0]['captions'][0][:80]}...")
            break


## 4. Modelle initialisieren

In [ ]:
class CaptioningModel:
    """Basis-Klasse für Captioning-Modelle"""

    def __init__(self, name: str):
        self.name = name
        self.device = device

    def generate_caption(self, image: Image.Image) -> str:
        raise NotImplementedError

    def generate_batch(self, images: List[Image.Image]) -> List[str]:
        return [self.generate_caption(img) for img in tqdm(images, desc=f"{self.name} generiert")]


class BLIP2Model(CaptioningModel):
    """BLIP-2 Modell mit Q-Former"""

    def __init__(self):
        super().__init__("BLIP-2")
        print(f"Lade {self.name}...")

        # BLIP-2 mit Flan-T5-XL (größere Version)
        model_name = "Salesforce/blip2-flan-t5-xl"

        self.processor = Blip2Processor.from_pretrained(model_name)
        self.model = Blip2ForConditionalGeneration.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16
        )

        print(f"✓ {self.name} geladen")

    def generate_caption(self, image: Image.Image, prompt: str = "") -> str:
        # Prompt für bessere Captions
        if not prompt:
            prompt = "A detailed description of this image:"

        inputs = self.processor(image, text=prompt, return_tensors="pt").to(
            self.device, torch.float16
        )

        with torch.no_grad():
            generated_ids = self.model.generate(
                **inputs,
                max_length=CONFIG['max_length'],
                num_beams=CONFIG['num_beams'],
                early_stopping=True
            )

        caption = self.processor.decode(generated_ids[0], skip_special_tokens=True)
        return caption.strip()


class CLIPCapModel(CaptioningModel):
    """CLIPCap Implementierung (vereinfacht)"""

    def __init__(self):
        super().__init__("CLIPCap")
        print(f"Lade {self.name}...")

        # Nutze BLIP als Alternative zu CLIPCap (einfacher verfügbar)
        # Für echte CLIPCap-Implementation benötigt man das originale Repo
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        self.model = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-image-captioning-large",
            torch_dtype=torch.float16
        ).to(self.device)

        print(f"✓ {self.name} (BLIP-Variante) geladen")

    def generate_caption(self, image: Image.Image) -> str:
        inputs = self.processor(image, return_tensors="pt").to(
            self.device, torch.float16
        )

        with torch.no_grad():
            generated_ids = self.model.generate(
                **inputs,
                max_length=CONFIG['max_length'],
                num_beams=CONFIG['num_beams']
            )

        caption = self.processor.decode(generated_ids[0], skip_special_tokens=True)
        return caption.strip()


class GPT4VModel(CaptioningModel):
    """GPT-4V über API"""

    def __init__(self, api_key: str = None):
        super().__init__("GPT-4V")

        if api_key:
            import openai
            self.client = openai.OpenAI(api_key=api_key)
            self.enabled = True
            print(f"✓ {self.name} API initialisiert")
        else:
            self.enabled = False
            print(f"⚠ {self.name} benötigt API-Key (übersprungen)")

    def generate_caption(self, image: Image.Image) -> str:
        if not self.enabled:
            return "GPT-4V not available (no API key)"

        import base64
        from io import BytesIO

        # Bild zu Base64
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()

        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",  # GPT-4V Nachfolger
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": "Provide a detailed, accessible description of this image suitable for visually impaired users. Focus on the main content, objects, people, actions, and context."
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/png;base64,{img_str}"
                                }
                            }
                        ]
                    }
                ],
                max_tokens=100
            )
            return response.choices[0].message.content.strip()

        except Exception as e:
            print(f"GPT-4V Fehler: {e}")
            return "Error generating caption"


# Modelle initialisieren
print("\n" + "="*50)
print("Initialisiere Modelle...")
print("="*50 + "\n")

models = {}

# BLIP-2 (Hauptmodell)
models['blip2'] = BLIP2Model()

# CLIPCap Alternative
models['clipcap'] = CLIPCapModel()

# GPT-4V (optional, benötigt API Key)
# Setze hier deinen OpenAI API Key ein:
OPENAI_API_KEY = None  # oder "sk-..."
models['gpt4v'] = GPT4VModel(api_key=OPENAI_API_KEY)

print("\n✓ Alle Modelle initialisiert")

## 5. Caption-Generierung

In [ ]:
def generate_captions_for_dataset(dataset_name: str, dataset_samples: List[Dict], models: Dict):
    """Generiert Captions für einen Datensatz mit allen Modellen"""

    results = []

    print(f"\n{'='*60}")
    print(f"Generiere Captions für {dataset_name.upper()}")
    print(f"{'='*60}\n")

    for model_name, model in models.items():
        print(f"\n--- {model.name} ---")

        for sample in tqdm(dataset_samples, desc=f"{model.name} auf {dataset_name}"):
            try:
                image = sample['image']
                if not isinstance(image, Image.Image):
                    image = Image.fromarray(image)

                # Caption generieren
                generated_caption = model.generate_caption(image)

                # Ergebnis speichern
                result = {
                    'dataset': dataset_name,
                    'image_id': sample.get('image_id', 'unknown'),
                    'model': model_name,
                    'generated_caption': generated_caption,
                    'reference_captions': sample.get('captions', []),
                }

                # VizWiz spezifische Felder
                if dataset_name == 'vizwiz':
                    result['question'] = sample.get('question', '')
                    result['answers'] = sample.get('answers', [])

                results.append(result)

            except Exception as e:
                print(f"Fehler bei Bild {sample.get('image_id', '?')}: {e}")
                continue

        # Zwischenspeicherung nach jedem Modell
        torch.cuda.empty_cache()

    return results


# Captions für alle Datensätze generieren
all_results = []

for dataset_name, dataset_samples in datasets.items():
    if len(dataset_samples) > 0:
        results = generate_captions_for_dataset(dataset_name, dataset_samples, models)
        all_results.extend(results)

# Ergebnisse als DataFrame
results_df = pd.DataFrame(all_results)

print(f"\n✓ Insgesamt {len(results_df)} Captions generiert")
print(f"\nÜbersicht:")
print(results_df.groupby(['dataset', 'model']).size())


Generiere Captions für COCO


--- BLIP-2 ---


BLIP-2 auf coco:   0%|          | 0/100 [00:00<?, ?it/s]


--- CLIPCap ---


CLIPCap auf coco:   0%|          | 0/100 [00:00<?, ?it/s]


--- GPT-4V ---


GPT-4V auf coco:   0%|          | 0/100 [00:00<?, ?it/s]


Generiere Captions für VIZWIZ


--- BLIP-2 ---


BLIP-2 auf vizwiz:   0%|          | 0/100 [00:00<?, ?it/s]

## 6. Automatische Evaluation (BLEU, METEOR, CIDEr, SPICE)

In [ ]:
class CaptionEvaluator:
    """Evaluiert Captions mit automatischen Metriken"""

    def __init__(self):
        self.scorers = {
            'BLEU': Bleu(4),
            'METEOR': Meteor(),
            'CIDEr': Cider(),
            # 'SPICE': Spice()  # Benötigt Java - optional
        }

    def prepare_data(self, results_df: pd.DataFrame):
        """Bereitet Daten für pycocoevalcap vor"""
        gts = {}  # Ground truth
        res = {}  # Results

        for idx, row in results_df.iterrows():
            img_id = f"{row['dataset']}_{row['image_id']}_{row['model']}"

            # References (ground truth)
            refs = row['reference_captions']
            if isinstance(refs, list) and len(refs) > 0:
                gts[img_id] = refs
            else:
                gts[img_id] = ['No reference available']

            # Generated caption
            res[img_id] = [row['generated_caption']]

        return gts, res

    def compute_metrics(self, gts: Dict, res: Dict) -> Dict:
        """Berechnet alle Metriken"""
        scores = {}

        for name, scorer in self.scorers.items():
            try:
                print(f"Berechne {name}...")
                score, _ = scorer.compute_score(gts, res)

                if isinstance(score, list):
                    # BLEU gibt 4 Werte zurück (1-4 grams)
                    for i, s in enumerate(score, 1):
                        scores[f'{name}-{i}'] = s
                else:
                    scores[name] = score

            except Exception as e:
                print(f"Fehler bei {name}: {e}")
                scores[name] = None

        return scores

    def evaluate_by_group(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Evaluiert nach Datensatz und Modell gruppiert"""

        evaluation_results = []

        groups = results_df.groupby(['dataset', 'model'])

        for (dataset, model), group in groups:
            print(f"\nEvaluiere {model} auf {dataset}...")

            # Daten vorbereiten
            gts, res = self.prepare_data(group)

            # Metriken berechnen
            scores = self.compute_metrics(gts, res)

            result = {
                'dataset': dataset,
                'model': model,
                'n_samples': len(group),
                **scores
            }

            evaluation_results.append(result)

        return pd.DataFrame(evaluation_results)


# Evaluation durchführen
evaluator = CaptionEvaluator()

print("\n" + "="*60)
print("AUTOMATISCHE EVALUATION")
print("="*60)

eval_results = evaluator.evaluate_by_group(results_df)

print("\n" + "="*60)
print("ERGEBNISSE")
print("="*60 + "\n")
print(eval_results.to_string(index=False))

# Speichern
eval_results.to_csv(f"{CONFIG['output_dir']}/automatic_metrics.csv", index=False)
print(f"\n✓ Ergebnisse gespeichert in {CONFIG['output_dir']}/automatic_metrics.csv")

## 7. WCAG-basierte qualitative Bewertung

In [ ]:
class WCAGEvaluator:
    """Bewertet Captions nach WCAG-Kriterien für Barrierefreiheit"""

    def __init__(self):
        self.criteria = {
            'verstaendlichkeit': {
                'name': 'Verständlichkeit',
                'description': 'Grammatikalisch korrekt, klar formuliert'
            },
            'informationsgehalt': {
                'name': 'Informationsgehalt',
                'description': 'Vollständige aber prägnante Wiedergabe der Bildinhalte'
            },
            'kontextadaequanz': {
                'name': 'Kontextadäquanz',
                'description': 'Semantisch konsistent, frei von Stereotypen'
            }
        }

    def analyze_caption_quality(self, caption: str) -> Dict:
        """Automatische Qualitätsanalyse (Heuristiken)"""

        scores = {}

        # Verständlichkeit
        word_count = len(caption.split())
        has_verb = any(word in caption.lower() for word in ['is', 'are', 'shows', 'contains', 'depicts'])

        verstaendlichkeit_score = 0
        if 5 <= word_count <= 30:  # Angemessene Länge
            verstaendlichkeit_score += 1
        if has_verb:  # Enthält Verb
            verstaendlichkeit_score += 1
        if caption[0].isupper() and caption.endswith('.'):  # Korrekte Interpunktion
            verstaendlichkeit_score += 1

        scores['verstaendlichkeit'] = min(verstaendlichkeit_score / 3, 1.0)

        # Informationsgehalt
        # Prüfe auf spezifische Objekte/Details
        has_specific_terms = any(len(word) > 4 for word in caption.split())
        informationsgehalt_score = 0.5 if has_specific_terms else 0.3
        if word_count >= 10:  # Ausreichend detailliert
            informationsgehalt_score += 0.3

        scores['informationsgehalt'] = min(informationsgehalt_score, 1.0)

        # Kontextadäquanz
        # Prüfe auf problematische Begriffe (vereinfachte Bias-Erkennung)
        problematic_terms = ['probably', 'seems', 'appears to be', 'might be']
        has_uncertainty = any(term in caption.lower() for term in problematic_terms)

        kontextadaequanz_score = 0.7 if not has_uncertainty else 0.4
        scores['kontextadaequanz'] = kontextadaequanz_score

        # Gesamt-Score
        scores['overall_wcag'] = np.mean(list(scores.values()))

        return scores

    def evaluate_dataset(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Evaluiert alle Captions nach WCAG"""

        wcag_scores = []

        for idx, row in tqdm(results_df.iterrows(), total=len(results_df), desc="WCAG-Evaluation"):
            caption = row['generated_caption']

            scores = self.analyze_caption_quality(caption)
            scores.update({
                'dataset': row['dataset'],
                'model': row['model'],
                'image_id': row['image_id']
            })

            wcag_scores.append(scores)

        return pd.DataFrame(wcag_scores)

    def aggregate_scores(self, wcag_df: pd.DataFrame) -> pd.DataFrame:
        """Aggregiert WCAG-Scores nach Modell und Datensatz"""

        agg_results = wcag_df.groupby(['dataset', 'model']).agg({
            'verstaendlichkeit': 'mean',
            'informationsgehalt': 'mean',
            'kontextadaequanz': 'mean',
            'overall_wcag': 'mean'
        }).reset_index()

        return agg_results


# WCAG Evaluation durchführen
wcag_evaluator = WCAGEvaluator()

print("\n" + "="*60)
print("WCAG-BASIERTE QUALITATIVE BEWERTUNG")
print("="*60 + "\n")

wcag_df = wcag_evaluator.evaluate_dataset(results_df)
wcag_summary = wcag_evaluator.aggregate_scores(wcag_df)

print("\nWCAG-Scores (aggregiert):")
print(wcag_summary.to_string(index=False))

# Speichern
wcag_df.to_csv(f"{CONFIG['output_dir']}/wcag_detailed.csv", index=False)
wcag_summary.to_csv(f"{CONFIG['output_dir']}/wcag_summary.csv", index=False)

print(f"\n✓ WCAG-Ergebnisse gespeichert")

## 8. Visualisierung und Beispiele

In [ ]:
# Metriken visualisieren
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Evaluation Metriken - Vergleich der Modelle', fontsize=16, fontweight='bold')

# BLEU-4
ax = axes[0, 0]
for dataset in eval_results['dataset'].unique():
    subset = eval_results[eval_results['dataset'] == dataset]
    ax.bar(subset['model'], subset['BLEU-4'], label=dataset, alpha=0.7)
ax.set_title('BLEU-4 Score')
ax.set_ylabel('Score')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# METEOR
ax = axes[0, 1]
for dataset in eval_results['dataset'].unique():
    subset = eval_results[eval_results['dataset'] == dataset]
    ax.bar(subset['model'], subset['METEOR'], label=dataset, alpha=0.7)
ax.set_title('METEOR Score')
ax.set_ylabel('Score')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# CIDEr
ax = axes[1, 0]
for dataset in eval_results['dataset'].unique():
    subset = eval_results[eval_results['dataset'] == dataset]
    ax.bar(subset['model'], subset['CIDEr'], label=dataset, alpha=0.7)
ax.set_title('CIDEr Score')
ax.set_ylabel('Score')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# WCAG Overall
ax = axes[1, 1]
for dataset in wcag_summary['dataset'].unique():
    subset = wcag_summary[wcag_summary['dataset'] == dataset]
    ax.bar(subset['model'], subset['overall_wcag'], label=dataset, alpha=0.7)
ax.set_title('WCAG Gesamt-Score')
ax.set_ylabel('Score')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f"{CONFIG['output_dir']}/metrics_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Visualisierung gespeichert")

In [ ]:
# Beispiel-Captions anzeigen
print("\n" + "="*80)
print("BEISPIEL-CAPTIONS")
print("="*80 + "\n")

for dataset_name in ['coco', 'flickr30k', 'vizwiz']:
    subset = results_df[results_df['dataset'] == dataset_name]

    if len(subset) == 0:
        continue

    print(f"\n{'='*80}")
    print(f"Dataset: {dataset_name.upper()}")
    print(f"{'='*80}\n")

    # Zeige erste 2 Beispiele
    for image_id in subset['image_id'].unique()[:2]:
        examples = subset[subset['image_id'] == image_id]

        print(f"\nBild ID: {image_id}")
        print("-" * 80)

        # Reference captions
        ref = examples.iloc[0]['reference_captions']
        if isinstance(ref, list) and len(ref) > 0:
            print(f"\nReferenz-Caption(s):")
            for i, r in enumerate(ref[:3], 1):  # Zeige max 3
                print(f"  {i}. {r}")

        # Generated captions
        print(f"\nGenerierte Captions:")
        for _, row in examples.iterrows():
            print(f"  [{row['model']}]: {row['generated_caption']}")

        print()

print("\n" + "="*80)
print("✓ Evaluation abgeschlossen")
print("="*80)

## 9. Export und Zusammenfassung

In [ ]:
# Kombinierte Ergebnistabelle erstellen
final_results = eval_results.merge(
    wcag_summary,
    on=['dataset', 'model'],
    how='outer'
)

# Sortieren
final_results = final_results.sort_values(['dataset', 'model'])

# Formatierung für bessere Lesbarkeit
numeric_cols = final_results.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col != 'n_samples':
        final_results[col] = final_results[col].round(4)

print("\n" + "="*100)
print("FINALE ERGEBNISTABELLE")
print("="*100 + "\n")
print(final_results.to_string(index=False))

# Als Excel exportieren
with pd.ExcelWriter(f"{CONFIG['output_dir']}/evaluation_results.xlsx") as writer:
    final_results.to_excel(writer, sheet_name='Summary', index=False)
    results_df.to_excel(writer, sheet_name='All_Captions', index=False)
    wcag_df.to_excel(writer, sheet_name='WCAG_Detailed', index=False)

print(f"\n✓ Excel-Datei erstellt: {CONFIG['output_dir']}/evaluation_results.xlsx")

# Zusammenfassung
print("\n" + "="*100)
print("ZUSAMMENFASSUNG")
print("="*100 + "\n")

print(f"Evaluierte Datensätze: {', '.join(results_df['dataset'].unique())}")
print(f"Getestete Modelle: {', '.join(results_df['model'].unique())}")
print(f"Gesamt Captions generiert: {len(results_df)}")
print(f"\nMetriken berechnet: BLEU-1 bis BLEU-4, METEOR, CIDEr")
print(f"WCAG-Kriterien: Verständlichkeit, Informationsgehalt, Kontextadäquanz")

print("\n" + "="*100)
print("✓ EVALUATION ERFOLGREICH ABGESCHLOSSEN")
print("="*100)

# Download-Link für Colab
print("\nDateien zum Download:")
from google.colab import files
import zipfile

# ============================================================
# CAPTION EXPORT MODULE - Füge dies als neue Zelle ein
# ============================================================

import pandas as pd
import json
from pathlib import Path
from typing import Dict, List
import os

class CaptionExporter:
    """Exportiert generierte Captions in verschiedenen Formaten"""

    def __init__(self, output_dir: str):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def export_all_formats(self, results_df: pd.DataFrame):
        """Exportiert Captions in allen Formaten"""
        print("\n" + "="*80)
        print("EXPORTIERE CAPTIONS IN ALLE FORMATE")
        print("="*80 + "\n")

        self.export_csv(results_df)
        self.export_excel(results_df)
        self.export_json(results_df)
        self.export_txt(results_df)
        self.export_html(results_df)
        self.export_model_summaries(results_df)

        print("\n✓ ALLE CAPTION-EXPORTS ABGESCHLOSSEN")

    def export_csv(self, results_df: pd.DataFrame):
        """Exportiert als CSV"""
        csv_path = self.output_dir / "all_generated_captions.csv"
        results_df.to_csv(csv_path, index=False, encoding='utf-8')
        print(f"✓ CSV: {csv_path}")

        compact_cols = ['dataset', 'model', 'image_id', 'generated_caption']
        compact_path = self.output_dir / "captions_compact.csv"
        results_df[compact_cols].to_csv(compact_path, index=False, encoding='utf-8')
        print(f"✓ Kompakte CSV: {compact_path}")

    def export_excel(self, results_df: pd.DataFrame):
        """Exportiert als Excel mit mehreren Sheets"""
        excel_path = self.output_dir / "captions_complete.xlsx"

        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            results_df.to_excel(writer, sheet_name='Alle_Captions', index=False)

            for model in results_df['model'].unique():
                model_data = results_df[results_df['model'] == model]
                sheet_name = f'{model}_Captions'[:31]
                model_data.to_excel(writer, sheet_name=sheet_name, index=False)

            for dataset in results_df['dataset'].unique():
                dataset_data = results_df[results_df['dataset'] == dataset]
                sheet_name = f'{dataset}_Captions'[:31]
                dataset_data.to_excel(writer, sheet_name=sheet_name, index=False)

            comparison_df = self.create_comparison_view(results_df)
            comparison_df.to_excel(writer, sheet_name='Vergleich_Side_by_Side', index=False)

        print(f"✓ Excel (mehrere Sheets): {excel_path}")

    def create_comparison_view(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Erstellt Side-by-Side Vergleichsansicht"""
        comparison_rows = []

        for (dataset, image_id), group in results_df.groupby(['dataset', 'image_id']):
            row = {'dataset': dataset, 'image_id': image_id}

            if 'reference_captions' in group.columns:
                refs = group.iloc[0]['reference_captions']
                if isinstance(refs, list) and len(refs) > 0:
                    row['reference_1'] = refs[0] if len(refs) > 0 else ''
                    row['reference_2'] = refs[1] if len(refs) > 1 else ''
                    row['reference_3'] = refs[2] if len(refs) > 2 else ''

            for _, item in group.iterrows():
                model_name = item['model']
                row[f'{model_name}_caption'] = item['generated_caption']

            comparison_rows.append(row)

        return pd.DataFrame(comparison_rows)

    def export_json(self, results_df: pd.DataFrame):
        """Exportiert als JSON"""
        json_data = {}

        for dataset in results_df['dataset'].unique():
            json_data[dataset] = {}
            dataset_data = results_df[results_df['dataset'] == dataset]

            for image_id in dataset_data['image_id'].unique():
                image_data = dataset_data[dataset_data['image_id'] == image_id]
                json_data[dataset][str(image_id)] = {
                    'reference_captions': [],
                    'generated_captions': {}
                }

                first_row = image_data.iloc[0]
                if 'reference_captions' in first_row:
                    refs = first_row['reference_captions']
                    if isinstance(refs, list):
                        json_data[dataset][str(image_id)]['reference_captions'] = refs

                for _, row in image_data.iterrows():
                    model = row['model']
                    caption = row['generated_caption']
                    json_data[dataset][str(image_id)]['generated_captions'][model] = caption

        json_path = self.output_dir / "captions_structured.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)

        print(f"✓ JSON: {json_path}")

    def export_txt(self, results_df: pd.DataFrame):
        """Exportiert als lesbares Text-Format"""
        txt_path = self.output_dir / "captions_readable.txt"

        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("GENERIERTE BILDUNTERSCHRIFTEN - ÜBERSICHT\n")
            f.write("="*80 + "\n\n")

            for dataset in sorted(results_df['dataset'].unique()):
                f.write(f"\n{'='*80}\n")
                f.write(f"DATASET: {dataset.upper()}\n")
                f.write(f"{'='*80}\n\n")

                dataset_data = results_df[results_df['dataset'] == dataset]

                for image_id in sorted(dataset_data['image_id'].unique())[:10]:  # Max 10 pro Dataset
                    image_data = dataset_data[dataset_data['image_id'] == image_id]

                    f.write(f"\n{'-'*80}\n")
                    f.write(f"Bild ID: {image_id}\n")
                    f.write(f"{'-'*80}\n\n")

                    first_row = image_data.iloc[0]
                    if 'reference_captions' in first_row:
                        refs = first_row['reference_captions']
                        if isinstance(refs, list) and len(refs) > 0:
                            f.write("REFERENZ-CAPTIONS:\n")
                            for i, ref in enumerate(refs[:5], 1):
                                f.write(f"  {i}. {ref}\n")
                            f.write("\n")

                    f.write("GENERIERTE CAPTIONS:\n")
                    for _, row in image_data.iterrows():
                        model = row['model']
                        caption = row['generated_caption']
                        f.write(f"  [{model:10s}]: {caption}\n")
                    f.write("\n")

        print(f"✓ TXT (lesbar): {txt_path}")

    def export_html(self, results_df: pd.DataFrame):
        """Exportiert als interaktive HTML-Seite"""
        html_path = self.output_dir / "captions_interactive.html"

        html = f"""<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="UTF-8">
    <title>Generierte Bildunterschriften</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px; background: #f5f5f5; }}
        h1 {{ color: #2c3e50; text-align: center; border-bottom: 3px solid #3498db; padding-bottom: 10px; }}
        .filter {{ background: white; padding: 15px; margin-bottom: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
        select, input {{ padding: 8px; margin: 5px; border: 1px solid #ddd; border-radius: 4px; }}
        .dataset-section {{ background: white; margin: 20px 0; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
        .dataset-title {{ color: #3498db; font-size: 24px; margin-bottom: 15px; border-bottom: 2px solid #ecf0f1; padding-bottom: 8px; }}
        .image-card {{ background: #fafafa; margin: 15px 0; padding: 15px; border-left: 4px solid #3498db; border-radius: 4px; }}
        .image-id {{ font-weight: bold; color: #2c3e50; margin-bottom: 10px; }}
        .reference-section {{ background: #e8f5e9; padding: 10px; margin: 10px 0; border-radius: 4px; border-left: 3px solid #4caf50; }}
        .caption-item {{ margin: 8px 0; padding: 10px; background: white; border-radius: 4px; border-left: 3px solid #9e9e9e; }}
        .model-name {{ display: inline-block; background: #3498db; color: white; padding: 3px 10px; border-radius: 3px; font-weight: bold; margin-right: 10px; min-width: 80px; text-align: center; }}
    </style>
</head>
<body>
    <h1>📊 Generierte Bildunterschriften</h1>
    <div class="filter">
        <select id="datasetFilter" onchange="filterContent()">
            <option value="all">Alle Datensätze</option>
"""

        for dataset in sorted(results_df['dataset'].unique()):
            html += f'            <option value="{dataset}">{dataset.upper()}</option>\n'

        html += """        </select>
        <input type="text" id="search" placeholder="Suche..." onkeyup="filterContent()">
    </div>
    <div id="content">
"""

        for dataset in sorted(results_df['dataset'].unique()):
            html += f'        <div class="dataset-section" data-dataset="{dataset}">\n'
            html += f'            <div class="dataset-title">📁 {dataset.upper()}</div>\n'

            dataset_data = results_df[results_df['dataset'] == dataset]

            for image_id in sorted(dataset_data['image_id'].unique())[:5]:  # Max 5 pro Dataset
                image_data = dataset_data[dataset_data['image_id'] == image_id]

                html += f'            <div class="image-card">\n'
                html += f'                <div class="image-id">🖼️ Bild ID: {image_id}</div>\n'

                first_row = image_data.iloc[0]
                if 'reference_captions' in first_row:
                    refs = first_row['reference_captions']
                    if isinstance(refs, list) and len(refs) > 0:
                        html += '                <div class="reference-section">\n'
                        html += '                    <strong>Referenz:</strong><br>\n'
                        for ref in refs[:3]:
                            html += f'                    • {ref}<br>\n'
                        html += '                </div>\n'

                for _, row in image_data.iterrows():
                    model = row['model']
                    caption = row['generated_caption']
                    html += f'                <div class="caption-item">\n'
                    html += f'                    <span class="model-name">{model.upper()}</span>\n'
                    html += f'                    {caption}\n'
                    html += '                </div>\n'

                html += '            </div>\n'

            html += '        </div>\n'

        html += """    </div>
    <script>
        function filterContent() {
            const datasetFilter = document.getElementById('datasetFilter').value;
            const searchText = document.getElementById('search').value.toLowerCase();

            document.querySelectorAll('.dataset-section').forEach(section => {
                const show = datasetFilter === 'all' || section.dataset.dataset === datasetFilter;
                section.style.display = show ? 'block' : 'none';
            });

            if (searchText) {
                document.querySelectorAll('.caption-item').forEach(item => {
                    const matches = item.textContent.toLowerCase().includes(searchText);
                    item.style.display = matches ? 'block' : 'none';
                });
            }
        }
    </script>
</body>
</html>"""

        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html)

        print(f"✓ HTML (interaktiv): {html_path}")

    def export_model_summaries(self, results_df: pd.DataFrame):
        """Erstellt separate Zusammenfassungen pro Modell"""
        summaries_dir = self.output_dir / "model_summaries"
        summaries_dir.mkdir(exist_ok=True)

        for model in results_df['model'].unique():
            model_data = results_df[results_df['model'] == model]

            model_csv = summaries_dir / f"{model}_captions.csv"
            model_data.to_csv(model_csv, index=False, encoding='utf-8')

        print(f"✓ Modell-Zusammenfassungen: {summaries_dir}")

def export_captions_comprehensive(results_df: pd.DataFrame, output_dir: str):
    """Exportiert alle Caption-Formate"""
    exporter = CaptionExporter(output_dir)
    exporter.export_all_formats(results_df)

    print(f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    CAPTION-EXPORT ABGESCHLOSSEN                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

Erstellt:
  ✓ all_generated_captions.csv
  ✓ captions_compact.csv
  ✓ captions_complete.xlsx (mehrere Sheets!)
  ✓ captions_structured.json
  ✓ captions_readable.txt
  ✓ captions_interactive.html (öffne im Browser!)
  ✓ model_summaries/

Alle Dateien in: {output_dir}
""")

print("✓ Caption Export Module geladen!")

# Caption-Export in alle Formate
from caption_export import export_captions_comprehensive

export_captions_comprehensive(
    results_df=results_df,
    output_dir=CONFIG['output_dir']
)

# Alle Ergebnisse zippen
zip_path = "/content/evaluation_results.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files_list in os.walk(CONFIG['output_dir']):
        for file in files_list:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.basename(file_path))

print(f"\n✓ Alle Ergebnisse als ZIP-Datei bereit")
print(f"  Download: {zip_path}")

# Optional: Automatischer Download
# files.download(zip_path)